## Step 1: Install Required Libraries

Run the following cell in Google Colab to install the required dependencies:

In [ ]:
!pip install llama-index openai langchain

## Step 2: Set Up the Environment
You need to set your OpenAI API key for LLM predictions and embeddings. Run:

In [ ]:
import os

# Set your OpenAI API Key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

Replace "your_openai_api_key" with your actual OpenAI API key. If you don’t have one, get it from OpenAI API Keys.

## Step 3: Implement the Single-Agent Agentic RAG Router
Now, run the following script:

In [ ]:
import os
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ListIndex,
    ServiceContext,
    LLMPredictor,
    OpenAIEmbedding,
    RouterQueryEngine,
    QueryEngineTool,
    ResponseSynthesizer
)
from langchain.llms import OpenAI
from llama_index.tools import ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

# Initialize the LLM predictor and service context
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0))
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=OpenAIEmbedding()
)

# Ensure you have some documents in Google Colab
# If needed, upload files using: from google.colab import files; files.upload()
docs_path = "/content/documents"  # Update this if needed
if not os.path.exists(docs_path):
    os.makedirs(docs_path)

# Load and index documents
documents = SimpleDirectoryReader(docs_path).load_data()
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
list_index = ListIndex.from_documents(documents, service_context=service_context)

# Create query engines for vector and list indices
vector_query_engine = vector_index.as_query_engine(similarity_top_k=5)
list_query_engine = list_index.as_query_engine(response_mode="summary")

# Define tools for the router
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    metadata=ToolMetadata(name='vector_search', description='Perform semantic search for relevant information.')
)
list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    metadata=ToolMetadata(name='summarizer', description='Summarize the content of the documents.')
)

# Initialize the router query engine with the defined tools
router_query_engine = RouterQueryEngine.from_defaults(
    selector=SubQuestionQueryEngine,
    tools=[vector_tool, list_tool],
    service_context=service_context,
    response_synthesizer=ResponseSynthesizer.from_defaults()
)

# Example query
query = "What are the key takeaways from the latest company report?"
response = router_query_engine.query(query)
print(response)

## Step 4: Upload Documents (If Needed)
If you want to upload documents manually in Google Colab, run this cell before executing the script:

In [ ]:
from google.colab import files

uploaded = files.upload()

# Move uploaded files to the `documents` directory
import shutil

docs_path = "/content/documents"
if not os.path.exists(docs_path):
    os.makedirs(docs_path)

for filename in uploaded.keys():
    shutil.move(filename, os.path.join(docs_path, filename))

print(f"Uploaded {len(uploaded.keys())} files successfully!")

## Step 5: Run the Query
Execute the router engine query in Google Colab to see the retrieval-augmented response.

In [ ]:
query = "Summarize the key insights from the uploaded documents."
response = router_query_engine.query(query)
print(response)

## What This Code Does

1.  Loads documents from Google Colab
2. Builds Vector & List indices
3. Implements a Router Query Engine
4. Supports semantic search & summarization
5. Queries fine-tuned RAG models

